# BERT: get a vector from word embeddings in any language

General steps:
1. INPUT: YOUR  OWN SENTENCE
2. CHOOSE A WORD OF YOUR INTEREST TO GET A VECTOR
3. OUTPUT: LIST OF WORDS FROM BERT-EMBEDDING

In [1]:
from collections import OrderedDict
#import logging; logging.basicConfig(level=logging.INFO)
import operator
import pdb
import sys
import torch
from transformers import *

I1228 16:26:15.837218  4592 file_utils.py:40] PyTorch version 1.0.1 available.


A list of BERT model names:
    
    English:
    bert-base-uncased, bert-large-uncased, bert-base-cased, bert-large-cased;
    
    102 languages (including English):
    bert-base-multilingual-uncased, bert-base-multilingual-cased; 
    
    Chinese language:
    bert-base-chinese.

If you have a task to extract NER or work with sentenses use 'cased' version.

In [2]:
class BertVector:
    
    def __init__(self):
        self.tokenized_text = tokenizer.tokenize(self.get_sent())
        
    def get_sent(self):
        print("Enter sentence:")
        self.sent = input()
        if (not self.sent.endswith(".")):
            self.sent += " ."
        return '[CLS] ' + self.sent + '[SEP]'

    def show_tokens(self):
        temp_str = ""
        for i in range(len(self.tokenized_text)):
            temp_str += "   " +  str(i) + ":" + self.tokenized_text[i]
        print(temp_str + '\n')

    def get_pos(self):
        try:
            print("Enter a number of token to get a word vector or 0 to quit:")
            self.masked_index = int(input())
            if isinstance(self.masked_index, int):
                if self.masked_index > 0:
                    return self.masked_index
                else:
                    raise SystemExit("Program closed.")
        except:
            print("Unexpected error:", sys.exc_info()[0])
            raise

            
# Loading pre-trained BERT model from online source
def main():
    # Choose a model type we wish to use
    #MODEL_TYPE = 'bert-base-multilingual-cased' 

    # When we use multilingual model due to non-latin languages inside BERT team recommends 
    # to pass this parameter: do_lower_case = false

    #tokenizer = BertTokenizer.from_pretrained(MODEL_TYPE, do_lower_case = False)
    #model = BertForMaskedLM.from_pretrained(MODEL_TYPE)
    #model.eval()
    
    while (True):
    
        bv = BertVector()

        #bv.tokenized_text = tokenizer.tokenize(bv.get_sent())
        bv.show_tokens()
        indexed_tokens = tokenizer.convert_tokens_to_ids(bv.tokenized_text)

        # Create segment tensors
        segments_ids = [0] * len(bv.tokenized_text)

        masked_index = len(bv.tokenized_text) - 2
        print(len(bv.tokenized_text), masked_index )
        bv.tokenized_text[masked_index] = "[MASK]"
        indexed_tokens[masked_index] = 103
        results_dict = {}

        # Convert inputs to PyTorch tensors
        tokens_tensor = torch.tensor([indexed_tokens])
        segments_tensors = torch.tensor([segments_ids])

        with torch.no_grad():
            predictions = model(tokens_tensor, segments_tensors)
            
        while True:
            bv.show_tokens()
            print("Enter any term position neighbor:")
            masked_index = bv.get_pos()
            results_dict = {}
            for i in range(len(predictions[0][0,masked_index])):
                tok = tokenizer.convert_ids_to_tokens([i])[0]
                results_dict[tok] = float(predictions[0][0,masked_index][i].tolist())

            k = 0
            hist_d = {}
            sorted_d = OrderedDict(sorted(results_dict.items(), key=lambda kv: kv[1], reverse=True))
            first = True
            max_val = 0
            for i in sorted_d:
                if (first):
                    max_val = sorted_d[i]
                    first = False
                val = round(float(sorted_d[i])/max_val,1)
                if (val in hist_d):
                    hist_d[val] += 1
                else:
                    hist_d[val] = 1
                k += 1
                if (k <= 20):
                    print(i,sorted_d[i])
            fp = open("top_k.txt","w")
            hist_d_sorted = OrderedDict(sorted(hist_d.items(), key=lambda kv: kv[0], reverse=False))
            for i in hist_d_sorted:
                fp.write(str(i) + " " + str(hist_d_sorted[i]) + "\n")
            fp.close()


In [3]:
MODEL_TYPE = 'bert-base-multilingual-cased' 

    # When we use multilingual model due to non-latin languages inside BERT team recommends 
    # to pass this parameter: do_lower_case = false

tokenizer = BertTokenizer.from_pretrained(MODEL_TYPE, do_lower_case = False)
model = BertForMaskedLM.from_pretrained(MODEL_TYPE)
model.eval()

I1228 16:26:18.192873  4592 tokenization_utils.py:379] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-multilingual-cased-vocab.txt from cache at C:\Users\Yuliya\.cache\torch\transformers\96435fa287fbf7e469185f1062386e05a075cadbf6838b74da22bf64b080bc32.99bcd55fc66f4f3360bc49ba472b940b8dcf223ea6a345deb969d607ca900729
I1228 16:26:18.939758  4592 configuration_utils.py:157] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-multilingual-cased-config.json from cache at C:\Users\Yuliya\.cache\torch\transformers\45629519f3117b89d89fd9c740073d8e4c1f0a70f9842476185100a8afe715d1.83b0fa3d7f1ac0e113ad300189a938c6f14d0588a4200f30eef109d0a047c484
I1228 16:26:18.951351  4592 configuration_utils.py:174] Model config {
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm(torch.Size([768]), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm(torch.Size([768]), eps=1e-12, elementwise_affine=True)
   

In [5]:
if __name__ == "__main__":
    main()


Enter sentence:
I like fish cooked in an oven.
   0:[CLS]   1:I   2:like   3:fish   4:co   5:##oked   6:in   7:an   8:ove   9:##n   10:.   11:[SEP]

12 10
   0:[CLS]   1:I   2:like   3:fish   4:co   5:##oked   6:in   7:an   8:ove   9:##n   10:[MASK]   11:[SEP]

Enter any term position neighbor:
Enter a number of token to get a word vector or 0 to quit:
3
fish 25.389249801635742
fishes 15.90997314453125
fishing 15.019346237182617
Fish 14.033843994140625
Fishes 13.935105323791504
food 13.413887023925781
##fish 12.747037887573242
it 12.60023307800293
meat 12.521782875061035
eat 12.061664581298828
you 11.883537292480469
birds 11.808743476867676
they 11.800027847290039
them 11.735713958740234
ones 11.636872291564941
eggs 11.597370147705078
water 11.238836288452148
that 11.172130584716797
some 10.927756309509277
things 10.738119125366211
   0:[CLS]   1:I   2:like   3:fish   4:co   5:##oked   6:in   7:an   8:ove   9:##n   10:[MASK]   11:[SEP]

Enter any term position neighbor:
Enter a number 

SystemExit: Program closed.